In [1]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm

/home/creambbq/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/creambbq/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
df = pd.read_stata('cornwell.dta')
df = df.assign(cte = 1)
df = df[['county', 'year', 'lcrmrte','lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']].assign(cte = 1)
df = df.dropna()
df = pd.concat([df, pd.get_dummies(df['year'])], axis = 1 )
x = df[['cte', 'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']]
x = pd.concat([x, pd.get_dummies(df['year'])], axis = 1 )
y = df['lcrmrte']

OLS COMÚN

In [3]:
pols = sm.OLS(y,x).fit()
pols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lcrmrte   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     74.49
Date:                Sat, 16 Apr 2022   Prob (F-statistic):          1.38e-105
Time:                        06:58:39   Log-Likelihood:                -276.51
No. Observations:                 630   AIC:                             577.0
Df Residuals:                     618   BIC:                             630.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cte           -1.8588      0.213     -8.741      0.000      -2.276      -1.441
lprbarr       -0.7195      0.037    -19.570      0.000      -0.792      -0.647
lprbconv      -0.5457      0.026    -20.694      0.000      -0.597      -0.494
lprbpris       0.2476      0.067      3.682      0.000       0.116       0.380
lavgsen       -0.0868      0.058     -1.498      0.135      -0.201       0.027
lpolpc         0.3660      0.030     12.189      0.000       0.307       0.425
81            -0.2235      0.052     -4.276      0.000      -0.326      -0.121
82            -0.2184      0.048     -4.594      0.000      -0.312      -0.125
83            -0.2670      0.048     -5.603      0.000      -0.361      -0.173
84            -0.3323      0.046     -7.222      0.000      -0.423      -0.242
85            -0.3015      0.046     -6.572      0.000      -0.392      -0.211
86            -0.2656      0.048     -5.533      0.000      -0.360      -0.171
87            -0.2505      0.050     -5.050      0.000      -0.348      -0.153
==============================================================================
Omnibus:                      105.104   Durbin-Watson:                   0.603
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              266.068
Skew:                          -0.858   Prob(JB):                     1.68e-58
Kurtosis:                       5.681   Cond. No.                     7.97e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.06e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Errores estandar robustos a la presencia de heterocedasticidad

In [4]:
pols_rob = sm.OLS(y,x).fit(cov_type='HC3')
pols_rob.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lcrmrte   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     6074.
Date:                Sat, 16 Apr 2022   Prob (F-statistic):               0.00
Time:                        06:58:39   Log-Likelihood:                -276.51
No. Observations:                 630   AIC:                             577.0
Df Residuals:                     618   BIC:                             630.4
Df Model:                          11                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
cte           -1.8588      0.392     -4.740      0.000      -2.627      -1.090
lprbarr       -0.7195      0.054    -13.244      0.000      -0.826      -0.613
lprbconv      -0.5457      0.040    -13.661      0.000      -0.624      -0.467
lprbpris       0.2476      0.088      2.811      0.005       0.075       0.420
lavgsen       -0.0868      0.076     -1.149      0.251      -0.235       0.061
lpolpc         0.3660      0.058      6.311      0.000       0.252       0.480
81            -0.2235      0.070     -3.197      0.001      -0.361      -0.086
82            -0.2184      0.068     -3.201      0.001      -0.352      -0.085
83            -0.2670      0.063     -4.206      0.000      -0.391      -0.143
84            -0.3323      0.071     -4.658      0.000      -0.472      -0.192
85            -0.3015      0.068     -4.462      0.000      -0.434      -0.169
86            -0.2656      0.065     -4.062      0.000      -0.394      -0.137
87            -0.2505      0.067     -3.715      0.000      -0.383      -0.118
==============================================================================
Omnibus:                      105.104   Durbin-Watson:                   0.603
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              266.068
Skew:                          -0.858   Prob(JB):                     1.68e-58
Kurtosis:                       5.681   Cond. No.                     7.97e+16
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 5.06e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Ejercicio Número 3

In [5]:
np.random.seed(1313)
n = 5 
T = 2
omg = np.array([[1,0], [0,4]])
muestras = 1000
data = {'i': [], 't': [], 'y': [], 'cte':[] , 'x': [], 'u': []}


for i in range(1,n+1) : 
    for t in range(1,T+1) : 
        data['cte'].append(1)
        data['i'].append(i)
        data['t'].append(t)
        x = np.random.uniform(1,20,1)
        x = x.astype(float)
        u = np.random.normal(0,omg.item(t-1,t-1))
        data['x'].append(x)
        data['u'].append(u)
        data['y'].append(1 + x + u)


data = pd.DataFrame(data=data)
y = data['y'].astype(float)
x = data[['cte', 'x']].astype(float)

In [6]:
reg_ols = sm.OLS(y,x).fit()

In [ ]:
u_hat = {'i': data['i'], 't': data['t'], 'u_jt': reg_ols.resid.tolist()}
u_hat = pd.DataFrame(data = u_hat)

In [56]:
omg_est = np.array([[0,0], [0,0]])
for l in range(0,n*T-1): 
    u_j = np.array([[u_hat['u_jt'][l]], [u_hat['u_jt'][l+1]]])
    omg_est = omg_est + u_j.dot(u_j.T)
omg_est
omg_inv = np.linalg.inv(omg_est)
x_j = np.asanyarray([[x[i], x[i+1]] for i in range(0,n*T,2)])
y_j = np.asanyarray([[y[i], y[i+1]] for i in range(0,n*T,2)])
var_est = np.zeros((2,2), float) #KXK cambiar nombre esta en realidad es la inversa
xT_omg_y = np.zeros((2,1), float) #kx1
for i in range(0,n): 
    temp = x_j[i].T.dot(omg_inv).dot(x_j[i])
    var_est = var_est + temp
    temp2 = x_j[i].T.dot(omg_inv).dot(y_j[i])
    xT_omg_y = xT_omg_y + temp2
    
xT_omg_y = xT_omg_y[1] #Me quedo solo con el primer vector porque numpy hace broadcasting 
beta_fgls = np.linalg.inv(var_est).dot(xT_omg_y)
beta_fgls #No son muy diferentes a los ols, me gusta 

In [123]:
test = np.array([[31.24440301, -0.95044039,0.,0.,0.,0.,0.,0.,0.,0.], 
         [-0.95044039, 24.67852892,0.,0.,0.,0.,0.,0.,0.,0.],
         [0.,0.,31.24440301, -0.95044039,0.,0.,0.,0.,0.,0.], 
         [0.,0.,-0.95044039, 24.67852892,0.,0.,0.,0.,0.,0.],
         [0.,0.,0.,0.,31.24440301, -0.95044039,0.,0.,0.,0.], 
         [0.,0.,0.,0.,-0.95044039, 24.67852892,0.,0.,0.,0.],
         [0.,0.,0.,0.,0.,0.,31.24440301, -0.95044039,0.,0.], 
         [0.,0.,0.,0.,0.,0.,-0.95044039, 24.67852892,0.,0.],
         [0.,0.,0.,0.,0.,0.,0.,0.,31.24440301, -0.95044039], 
         [0.,0.,0.,0.,0.,0.,0.,0.,-0.95044039, 24.67852892]])
test = pd.DataFrame(test)

In [125]:
y = data['y'].astype(float)
x = data[['cte', 'x']].astype(float)

In [130]:
fgls = sm.GLS(y,x,test).fit()
fgls.summary()

/home/creambbq/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.892
Model:                            GLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     66.04
Date:                Sat, 16 Apr 2022   Prob (F-statistic):           3.90e-05
Time:                        09:46:41   Log-Likelihood:                -20.246
No. Observations:                  10   AIC:                             44.49
Df Residuals:                       8   BIC:                             45.10
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cte            2.2509      1.846      1.219      0.257      -2.007       6.508
x              0.9857      0.121      8.127      0.000       0.706       1.265
==============================================================================
Omnibus:                        9.914   Durbin-Watson:                   1.765
Prob(Omnibus):                  0.007   Jarque-Bera (JB):                4.200
Skew:                           1.371   Prob(JB):                        0.122
Kurtosis:                       4.602   Cond. No.                         44.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [131]:
beta_fgls

array([2.25092463, 0.98571238])